# Extração de Informação dos Corpora

Este notebook extrai informações dos corpora compilados, utilizando recursos de níveis (morfo)sintático, de forma que:
- Carrega e estrutura os dados dos corpora em dicionário
- Exibe estatísticas acerca da estrutura construída
- Implementa um sistema simples de perguntas e respostas sobre os corpora
- Apresenta comentários acerca dos resultados obtidos

## Configurações Iniciais

Primeiramente, fazer o ajuste no path do Python para a raíz do projeto:

In [ ]:
import os
os.chdir('..')

---------------------------

## Processamento dos Corpora

Carregar a bliblioteca de normalização de texto:

In [ ]:
from nlputils.lexical.normalizer import Normalizer

Instanciar um objeto para fazer as normalizações necessárias:

In [ ]:
normalizer = Normalizer()

Implementar uma rotina para auxiliar o carregamento dos dados e evitar replicação de código.

A função ```load_sentences``` recebe como entrada a string indicando o path do diretório do córpus a ser usado, retornando a lista contendo todas as sentenças desse córpus:

In [ ]:
def load_sentences(corpus_dir):   
    all_sentences = []
    for file_path in os.listdir(corpus_dir):
        with open(corpus_dir + file_path) as fp:
            for line in fp:
                # line = normalizer.to_lowercase(line)
                sentences = normalizer.tokenize_sentences(line)
                # sentences = [normalizer.tokenize_words(sent) for sent in sentences]
                all_sentences.extend(sentences)
    return all_sentences

---------------

## Carregando as Sentenças dos Corpora

Carregar todas as sentenças de cada córpus e mesclar tudo em uma só lista:

In [ ]:
sentences = []

Carregar cada um dos córpus e concatenar na lista:

In [ ]:
sentences.extend(load_sentences('data/corpora/tecnologia/'))

In [ ]:
sentences.extend(load_sentences('data/corpora/mercado/'))

In [ ]:
sentences.extend(load_sentences('data/corpora/telefonia/'))

In [ ]:
sentences.extend(load_sentences('data/corpora/saude/'))

---------------------------------

## Extração de Informações 

Importar a biblioteca ```Syntax``` para fazer o processamento nos corpora:

In [ ]:
from nlputils.syntax.parser import Parser

Instanciar um objeto para fazer o parse:

In [ ]:
parser = Parser()

Vamos transformar as senteças de cada corpus na seguinte estrutura de dados:
```
{
    “verbo_lematizado_1”: [(Sujeito_1, Objeto_1), (Sujeito_2, None), ..., (Sujeito_n, Objeto_n)], 
    ... 
}
```

In [ ]:
data = {}
for sentence in sentences:
    svo_list = parser.get_SVO(sentence)
    for subj, verb, obj in svo_list:
        if verb.lemma_ in data:
            data[verb.lemma_].append((subj, obj))
        else:
            data[verb.lemma_]= [(subj, obj)]

Logo abaixo está o resultado do dicionário:

In [ ]:
import pprint

pp = pprint.PrettyPrinter(indent=2)

pp.pprint(data)

### Estatísticas em cima do dicionário ```data```

* Qual verbo tem a maior lista de sujeitos e objetos?

In [ ]:
greatest = [[len(data[d]), d] for d in data]
greatest = max(greatest)
greatest

* Há algum verbo sem objetos? Mostre alguns.

In [ ]:
smaller = [[len(data[d]), d] for d in data]
smaller = min(smaller)
smaller

---------------------------

## Sistema básico de perguntas e respostas utilizando o dicionário ```data```

Importar a biblioteca ```Morphosyntax``` para fazer o processamento nos dados:

In [ ]:
from nlputils.morphosyntax.tagger import Tagger

Instanciar um objeto para fazer a etiquetagem:

In [ ]:
tagger = Tagger()

...

In [ ]:
question = input('Digite uma pergunta: ')

import spacy
nlp = spacy.load('pt_core_news_sm')

tagged_text = nlp(question)
verb = ''
for token in tagged_text:
    if token.pos_ == 'VERB':
        if token.lemma_ in data.keys():
            verb = token.lemma_
try:
    for text in data[verb]:
        print('Resposta: {} {} {}'.format(text[0], verb, text[1]))
except KeyError:
    print('A base de dados do dicionário não consegue responder esta pergunta')
    